# Doc 8 Policy Combination Analysis

In [1]:
import numpy as np
import pandas as pd
import pickle

## 0 Basic data

### 0.1 Weight projection

In [2]:
# weight data
weight = pd.read_csv("data/weights_2016_2024.csv")
weight = weight[['year', 'ICEV & others', 'EV']]

from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

# linear function
def linear_func(x, a, b):
    return a * x + b

# fit data
## icev
x_icev = weight.loc[1:]['year'].values
y_icev = weight.loc[1:]['ICEV & others'].values
params_icev, params_covariance_icev = curve_fit(linear_func, x_icev, y_icev)
r2_icev = r2_score(y_icev, linear_func(x_icev, *params_icev))

## ev
x_ev = weight.loc[1:]['year'].values
y_ev = weight.loc[1:]['EV'].values
params_ev, params_covariance_ev = curve_fit(linear_func, x_ev, y_ev)
r2_ev = r2_score(y_ev, linear_func(x_ev, *params_ev))

In [3]:
from scipy import stats
# icev
slope_icev, intercept_icev, r_value_icev, p_value_icev, std_err_icev = stats.linregress(x_icev, y_icev)
# ev
slope_ev, intercept_ev, r_value_ev, p_value_ev, std_err_ev = stats.linregress(x_ev, y_ev)

p_value_icev, p_value_ev

(1.739188522835111e-05, 3.877033328431473e-06)

In [4]:
# projection
weight_proj = weight.copy()
for y in np.arange(2025, 2037, 1):
    weight_proj.loc[weight_proj.shape[0]] = [y, linear_func(y, *params_icev), linear_func(y, *params_ev)]
weight_proj = weight_proj[weight_proj['year'] >= 2023].reset_index(drop=True)

ev_2024 = weight_proj[weight_proj['year'] == 2024]['EV'].values
icev_2024 = weight_proj[weight_proj['year'] == 2024]['ICEV & others'].values
ev_2035 = weight_proj[weight_proj['year'] == 2035]['EV'].values
icev_2035 = weight_proj[weight_proj['year'] == 2035]['ICEV & others'].values

### 0.2 Weight adjust EFs

In [5]:
# EF data
with open("data/vehicle_ef_cltc.pkl", 'rb') as f:
    ef_cltc = pickle.load(f)

# set 2024 as the base
base_ef_cltc = ef_cltc.loc[2024]
# base weight
weight_2024_icev = weight_proj[weight_proj['year'] == 2024]['ICEV & others']
weight_2024_ev = weight_proj[weight_proj['year'] == 2024]['EV']

In [6]:
def weight_adjust(EF, weight, assumption, weight_ref):
    """
    Adjust calculated EF according to weight.
    """
    if assumption == 'linear':
        a = 1
    elif assumption == 'power':
        a = 1.9
    return EF * (weight/weight_ref)**(1/a)

In [7]:
proj_ef_cltc = weight_proj.copy()
proj_ef_cltc['ICEV EF linear'] = proj_ef_cltc.apply(lambda x: weight_adjust(base_ef_cltc['ICEV EF linear'], x['ICEV & others'], 'linear', weight_2024_icev), axis=1)
proj_ef_cltc['ICEV EF power'] = proj_ef_cltc.apply(lambda x: weight_adjust(base_ef_cltc['ICEV EF power'], x['ICEV & others'], 'power', weight_2024_icev), axis=1)
proj_ef_cltc['EV weak EF linear'] = proj_ef_cltc.apply(lambda x: weight_adjust(base_ef_cltc['EV weak EF linear'], x['EV'], 'linear', weight_2024_ev), axis=1)
proj_ef_cltc['EV weak EF power'] = proj_ef_cltc.apply(lambda x: weight_adjust(base_ef_cltc['EV weak EF power'], x['EV'], 'power', weight_2024_ev), axis=1)
proj_ef_cltc['EV strong EF linear'] = proj_ef_cltc.apply(lambda x: weight_adjust(base_ef_cltc['EV strong EF linear'], x['EV'], 'linear', weight_2024_ev), axis=1)
proj_ef_cltc['EV strong EF power'] = proj_ef_cltc.apply(lambda x: weight_adjust(base_ef_cltc['EV strong EF power'], x['EV'], 'power', weight_2024_ev), axis=1)

### 0.3 Penetration rate adjustment

In [8]:
pen_rate_range = np.arange(0,100,1) / 100 

In [9]:
pen_proj_ef_cltc = pd.DataFrame(columns=['year', 'weight icev', 'weight ev', 'penetration', 'ev weak ef', 'ev strong ef'])
for pen in pen_rate_range:
    for index, row in proj_ef_cltc.iterrows():
        year = row['year']
        weight_icev = row['ICEV & others']
        weight_ev = row['EV']
        # weak reg
        ev_weak_ef_linear = row['ICEV EF linear'] * (1-pen) + row['EV weak EF linear'] * pen
        ev_weak_ef_power = row['ICEV EF power'] * (1-pen) + row['EV weak EF power'] * pen
        ev_weak_ef = (ev_weak_ef_linear + ev_weak_ef_power) / 2
        # strong reg
        ev_strong_ef_linear = row['ICEV EF linear'] * (1-pen) + row['EV strong EF linear'] * pen
        ev_strong_ef_power = row['ICEV EF power'] * (1-pen) + row['EV strong EF power'] * pen
        ev_strong_ef = (ev_strong_ef_linear + ev_strong_ef_power) / 2
        # add row
        pen_proj_ef_cltc.loc[pen_proj_ef_cltc.shape[0]] = [year, weight_icev, weight_ev, pen, ev_weak_ef, ev_strong_ef]
# strong reg. promotion
pen_proj_ef_cltc['ef'] = (pen_proj_ef_cltc['ev weak ef'] + pen_proj_ef_cltc['ev strong ef']) / 2
pen_proj_ef_cltc['ef 0.6 strong'] = 0.4 * pen_proj_ef_cltc['ev weak ef'] + 0.6 * pen_proj_ef_cltc['ev strong ef']
pen_proj_ef_cltc['ef 0.7 strong'] = 0.3 * pen_proj_ef_cltc['ev weak ef'] + 0.7 * pen_proj_ef_cltc['ev strong ef']
pen_proj_ef_cltc['ef 0.8 strong'] = 0.2 * pen_proj_ef_cltc['ev weak ef'] + 0.8 * pen_proj_ef_cltc['ev strong ef']
pen_proj_ef_cltc['ef 0.9 strong'] = 0.1 * pen_proj_ef_cltc['ev weak ef'] + 0.9 * pen_proj_ef_cltc['ev strong ef']
pen_proj_ef_cltc['ef 1.0 strong'] = pen_proj_ef_cltc['ev strong ef']

In [10]:
# EF difference
pen_proj_ef_inc_cltc = pen_proj_ef_cltc.copy()
ef_2024_cltc = pen_proj_ef_cltc[(pen_proj_ef_cltc['year'] == 2024) & (pen_proj_ef_cltc['penetration'] == 0.09)]
pen_proj_ef_inc_cltc["ef"] = (pen_proj_ef_inc_cltc["ef"] - ef_2024_cltc['ef'].values[0]) / ef_2024_cltc['ef'].values[0]
pen_proj_ef_inc_cltc["ef 0.6 strong"] = (pen_proj_ef_inc_cltc["ef 0.6 strong"] - ef_2024_cltc['ef'].values[0]) / ef_2024_cltc['ef'].values[0]
pen_proj_ef_inc_cltc["ef 0.7 strong"] = (pen_proj_ef_inc_cltc["ef 0.7 strong"] - ef_2024_cltc['ef'].values[0]) / ef_2024_cltc['ef'].values[0]
pen_proj_ef_inc_cltc["ef 0.8 strong"] = (pen_proj_ef_inc_cltc["ef 0.8 strong"] - ef_2024_cltc['ef'].values[0]) / ef_2024_cltc['ef'].values[0]
pen_proj_ef_inc_cltc["ef 0.9 strong"] = (pen_proj_ef_inc_cltc["ef 0.9 strong"] - ef_2024_cltc['ef'].values[0]) / ef_2024_cltc['ef'].values[0]
pen_proj_ef_inc_cltc["ef 1.0 strong"] = (pen_proj_ef_inc_cltc["ef 1.0 strong"] - ef_2024_cltc['ef'].values[0]) / ef_2024_cltc['ef'].values[0]

### 0.4 Lightweighting

In [11]:
# weight data
weight = pd.read_csv("data/weights_2016_2024.csv")
weight_2024 = weight.iloc[-1]
weight_2024_icev = weight_2024['ICEV & others']
weight_2024_ev = weight_2024['EV']
weight_2035_icev = weight_proj[weight_proj['year']==2035]['ICEV & others'].values[0]

def weight_adjust(EF, weight, assumption, weight_ref):
    """
    Adjust calculated EF according to weight.
    """
    if assumption == 'linear':
        a = 1
    elif assumption == 'power':
        a = 1.9
    return EF * (weight/weight_ref)**(1/a)

In [12]:
weight_inc_range = np.arange(-100, 401, 10)

weight_adjust_ef_cltc = pd.DataFrame(columns=['weight inc', 'ICEV EF linear', 'ICEV EF power', 'EV weak EF linear', 'EV weak EF power', 'EV strong EF linear', 'EV strong EF power'])
for inc in weight_inc_range:
    icev_ef_linear = weight_adjust(base_ef_cltc['ICEV EF linear'], weight_2035_icev, 'linear', weight_2024_icev)
    icev_ef_power = weight_adjust(base_ef_cltc['ICEV EF power'], weight_2035_icev, 'power', weight_2024_icev)
    ev_weak_ef_linear = weight_adjust(base_ef_cltc['EV weak EF linear'], weight_2024_ev+inc, 'linear', weight_2024_ev)
    ev_weak_ef_power = weight_adjust(base_ef_cltc['EV weak EF power'], weight_2024_ev+inc, 'power', weight_2024_ev)
    ev_strong_ef_linear = weight_adjust(base_ef_cltc['EV strong EF linear'], weight_2024_ev+inc, 'linear', weight_2024_ev)
    ev_strong_ef_power = weight_adjust(base_ef_cltc['EV strong EF power'], weight_2024_ev+inc, 'power', weight_2024_ev)
    weight_adjust_ef_cltc.loc[weight_adjust_ef_cltc.shape[0]] = [inc, icev_ef_linear, icev_ef_power, ev_weak_ef_linear, ev_weak_ef_power, ev_strong_ef_linear, ev_strong_ef_power]

In [13]:
pen_rate_range = np.arange(0,100,1) / 100 

In [14]:
pen_weight_adjust_ef_cltc = pd.DataFrame(columns=['weight inc', 'penetration', 'ev weak ef', 'ev strong ef'])
for pen in pen_rate_range:
    for index, row in weight_adjust_ef_cltc.iterrows():
        weight_inc = row['weight inc']
        # weak reg
        ev_weak_ef_linear = row['ICEV EF linear'] * (1-pen) + row['EV weak EF linear'] * pen
        ev_weak_ef_power = row['ICEV EF power'] * (1-pen) + row['EV weak EF power'] * pen
        ev_weak_ef = (ev_weak_ef_linear + ev_weak_ef_power) / 2
        # strong reg
        ev_strong_ef_linear = row['ICEV EF linear'] * (1-pen) + row['EV strong EF linear'] * pen
        ev_strong_ef_power = row['ICEV EF power'] * (1-pen) + row['EV strong EF power'] * pen
        ev_strong_ef = (ev_strong_ef_linear + ev_strong_ef_power) / 2
        # add row
        pen_weight_adjust_ef_cltc.loc[pen_weight_adjust_ef_cltc.shape[0]] = [weight_inc, pen, ev_weak_ef, ev_strong_ef]
# strong reg. promotion
pen_weight_adjust_ef_cltc['ef'] = (pen_weight_adjust_ef_cltc['ev weak ef'] + pen_weight_adjust_ef_cltc['ev strong ef']) / 2
pen_weight_adjust_ef_cltc['ef 0.6 strong'] = 0.4 * pen_weight_adjust_ef_cltc['ev weak ef'] + 0.6 * pen_weight_adjust_ef_cltc['ev strong ef']
pen_weight_adjust_ef_cltc['ef 0.7 strong'] = 0.3 * pen_weight_adjust_ef_cltc['ev weak ef'] + 0.7 * pen_weight_adjust_ef_cltc['ev strong ef']
pen_weight_adjust_ef_cltc['ef 0.8 strong'] = 0.2 * pen_weight_adjust_ef_cltc['ev weak ef'] + 0.8 * pen_weight_adjust_ef_cltc['ev strong ef']
pen_weight_adjust_ef_cltc['ef 0.9 strong'] = 0.1 * pen_weight_adjust_ef_cltc['ev weak ef'] + 0.9 * pen_weight_adjust_ef_cltc['ev strong ef']
pen_weight_adjust_ef_cltc['ef 1.0 strong'] = pen_weight_adjust_ef_cltc['ev strong ef']

In [15]:
# EF difference
pen_weight_adjust_ef_inc_cltc = pen_weight_adjust_ef_cltc.copy()
ef_2024_cltc = pen_proj_ef_cltc[(pen_proj_ef_cltc['year'] == 2024) & (pen_proj_ef_cltc['penetration'] == 0.09)]
pen_weight_adjust_ef_inc_cltc["ef"] = (pen_weight_adjust_ef_inc_cltc["ef"] - ef_2024_cltc['ef'].values[0]) / ef_2024_cltc['ef'].values[0]
pen_weight_adjust_ef_inc_cltc["ef 0.6 strong"] = (pen_weight_adjust_ef_inc_cltc["ef 0.6 strong"] - ef_2024_cltc['ef'].values[0]) / ef_2024_cltc['ef'].values[0]
pen_weight_adjust_ef_inc_cltc["ef 0.7 strong"] = (pen_weight_adjust_ef_inc_cltc["ef 0.7 strong"] - ef_2024_cltc['ef'].values[0]) / ef_2024_cltc['ef'].values[0]
pen_weight_adjust_ef_inc_cltc["ef 0.8 strong"] = (pen_weight_adjust_ef_inc_cltc["ef 0.8 strong"] - ef_2024_cltc['ef'].values[0]) / ef_2024_cltc['ef'].values[0]
pen_weight_adjust_ef_inc_cltc["ef 0.9 strong"] = (pen_weight_adjust_ef_inc_cltc["ef 0.9 strong"] - ef_2024_cltc['ef'].values[0]) / ef_2024_cltc['ef'].values[0]
pen_weight_adjust_ef_inc_cltc["ef 1.0 strong"] = (pen_weight_adjust_ef_inc_cltc["ef 1.0 strong"] - ef_2024_cltc['ef'].values[0]) / ef_2024_cltc['ef'].values[0]

---

## 1 Scenario 1: EV pen growth

In [19]:
pen_proj_ef_inc_cltc[(pen_proj_ef_inc_cltc['year']==2035) & (pen_proj_ef_inc_cltc['ef'] <= -0.27)].iloc[0]['penetration']

0.78

## 2 Scenario 2: EV pen growth + strong reg.

In [18]:
pen_proj_ef_inc_cltc[(pen_proj_ef_inc_cltc['year']==2035) & (pen_proj_ef_inc_cltc['ef 0.6 strong'] <= -0.27)].iloc[0]['penetration']

0.63

In [20]:
pen_proj_ef_inc_cltc[(pen_proj_ef_inc_cltc['year']==2035) & (pen_proj_ef_inc_cltc['ef 0.7 strong'] <= -0.27)].iloc[0]['penetration']

0.52

In [ ]:
pen_proj_ef_inc_cltc[(pen_proj_ef_inc_cltc['year']==2035) & (pen_proj_ef_inc_cltc['ef 0.8 strong'] <= -0.27)].iloc[0]['penetration']

0.45

In [22]:
pen_proj_ef_inc_cltc[(pen_proj_ef_inc_cltc['year']==2035) & (pen_proj_ef_inc_cltc['ef 0.9 strong'] <= -0.27)].iloc[0]['penetration']

0.39

In [23]:
pen_proj_ef_inc_cltc[(pen_proj_ef_inc_cltc['year']==2035) & (pen_proj_ef_inc_cltc['ef 1.0 strong'] <= -0.27)].iloc[0]['penetration']

0.35

## 3 Scenario 3: EV pen growth + lightweighting

In [26]:
pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == 300) & (pen_weight_adjust_ef_inc_cltc['ef'] <= -0.27)].iloc[0]['penetration']

0.74

In [27]:
pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == 200) & (pen_weight_adjust_ef_inc_cltc['ef'] <= -0.27)].iloc[0]['penetration']

0.71

In [28]:
pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == 100) & (pen_weight_adjust_ef_inc_cltc['ef'] <= -0.27)].iloc[0]['penetration']

0.68

In [30]:
pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == 0) & (pen_weight_adjust_ef_inc_cltc['ef'] <= -0.27)].iloc[0]['penetration']

0.65

In [31]:
pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == -100) & (pen_weight_adjust_ef_inc_cltc['ef'] <= -0.27)].iloc[0]['penetration']

0.62

## Scenario 4: EV pen growth + lightweighting + strong reg.

In [36]:
(pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == 300) & (pen_weight_adjust_ef_inc_cltc['ef 0.6 strong'] <= -0.27)].iloc[0]['penetration'],  
pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == 200) & (pen_weight_adjust_ef_inc_cltc['ef 0.6 strong'] <= -0.27)].iloc[0]['penetration'],  
pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == 100) & (pen_weight_adjust_ef_inc_cltc['ef 0.6 strong'] <= -0.27)].iloc[0]['penetration'],  
pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == 0) & (pen_weight_adjust_ef_inc_cltc['ef 0.6 strong'] <= -0.27)].iloc[0]['penetration'],  
pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == -100) & (pen_weight_adjust_ef_inc_cltc['ef 0.6 strong'] <= -0.27)].iloc[0]['penetration'])

(0.61, 0.59, 0.57, 0.55, 0.54)

In [37]:
(pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == 300) & (pen_weight_adjust_ef_inc_cltc['ef 0.7 strong'] <= -0.27)].iloc[0]['penetration'],  
pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == 200) & (pen_weight_adjust_ef_inc_cltc['ef 0.7 strong'] <= -0.27)].iloc[0]['penetration'],  
pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == 100) & (pen_weight_adjust_ef_inc_cltc['ef 0.7 strong'] <= -0.27)].iloc[0]['penetration'],  
pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == 0) & (pen_weight_adjust_ef_inc_cltc['ef 0.7 strong'] <= -0.27)].iloc[0]['penetration'],  
pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == -100) & (pen_weight_adjust_ef_inc_cltc['ef 0.7 strong'] <= -0.27)].iloc[0]['penetration'])

(0.51, 0.5, 0.49, 0.48, 0.47)

In [ ]:
pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == 100) & (pen_weight_adjust_ef_inc_cltc['ev strong ef'] <= -0.27)].iloc[0]

weight inc      100.000000
penetration       0.380000
ev weak ef        0.040175
ev strong ef     -0.274388
ef               -0.110101
Name: 1958, dtype: float64

In [64]:
pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == 0) & (pen_weight_adjust_ef_inc_cltc['ev strong ef'] <= -0.27)].iloc[0]

weight inc      0.000000
penetration     0.380000
ev weak ef      0.023258
ev strong ef   -0.275008
ef             -0.119232
Name: 1948, dtype: float64

In [65]:
pen_weight_adjust_ef_inc_cltc[(pen_weight_adjust_ef_inc_cltc['weight inc'] == -100) & (pen_weight_adjust_ef_inc_cltc['ev strong ef'] <= -0.27)].iloc[0]

weight inc     -100.000000
penetration       0.380000
ev weak ef        0.006182
ev strong ef     -0.275634
ef               -0.128449
Name: 1938, dtype: float64